In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import requests
import zipfile as zf

In [9]:
!rm -rf /content/withoutProcessing

zf.ZipFile('/content/drive/MyDrive/withoutProcessing.zip').extractall(path='/content', members=None, pwd=None)

In [94]:
!pip install tensorflow_addons split_folders

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [104]:
import keras
import keras.backend as K
import tensorflow_addons as tfa
from keras import layers
import tensorflow as tf
import numpy as np

from keras.applications import DenseNet201
from keras.applications.densenet import preprocess_input

import random
import os
from glob import glob
from skimage.io import imread
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from keras.callbacks import TensorBoard
from skimage.transform import resize
from sklearn.metrics import accuracy_score, cohen_kappa_score, precision_score, confusion_matrix

In [19]:
# Instancia da rede
denseNet201 = DenseNet201(
    include_top=False,
    weights="imagenet",
    input_shape=(250, 250, 3),
    classes=5,
)

In [14]:
shape=(250, 250)

In [105]:
def recall(y_true, y_pred):
    y_true = K.ones_like(y_true)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    all_positives = K.sum(K.round(K.clip(y_true, 0, 1)))

    recall = true_positives / (all_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    y_true = K.ones_like(y_true)
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))

    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [106]:
from tensorflow_addons.metrics.cohens_kappa import CohenKappa
from tensorflow_addons.metrics.f_scores import F1Score
from keras import metrics

metrics = [
    "accuracy",
    precision,
    tfa.metrics.CohenKappa(
        num_classes= 5,
        name = 'kappa'
    ),
]


In [76]:
import splitfolders

!rm -rf /content/output

path = '/content/withoutProcessing'
# drive_path = '/content/drive/MyDrive/output'
splitfolders.ratio(
    input=path,
    # output=drive_path,
    ratio=(0.75, 0.05, 0.2)
)

Copying files: 10000 files [00:01, 8794.18 files/s]


In [77]:
from tensorflow.keras.utils import image_dataset_from_directory

batch_size=32

def readImages(train_path, test_path, validation_path):
    
    train_dataset = image_dataset_from_directory(
        train_path,
        labels='inferred',
        image_size=shape,
        seed=42,
        batch_size=batch_size
    )

    test_dataset = image_dataset_from_directory(
        test_path,
        labels='inferred',
        image_size=shape,
        seed=42,
        batch_size=batch_size
    )

    validation_dataset = image_dataset_from_directory(
        validation_path,
        labels='inferred',
        image_size=shape,
        seed=42,
        batch_size=batch_size
    )

    train_dataset = train_dataset.prefetch(buffer_size = tf.data.AUTOTUNE)
    test_dataset = test_dataset.prefetch(buffer_size = tf.data.AUTOTUNE)
    validation_dataset = validation_dataset.prefetch(buffer_size = tf.data.AUTOTUNE)

    return train_dataset, test_dataset, validation_dataset

In [78]:
test_path = "/content/output/test"
train_path = "/content/output/train"
validation_path = "/content/output/val"

train_dataset, test_dataset, validation_dataset = readImages(train_path, test_path, validation_path)

Found 7500 files belonging to 5 classes.
Found 2000 files belonging to 5 classes.
Found 500 files belonging to 5 classes.


In [79]:
from keras.layers.serialization import activation
def buildModel(model: keras.Model):
    model.trainable = False

    global_average_layer = keras.layers.GlobalAveragePooling2D()

    camada_total_connect1 = keras.layers.Dense(50, activation="relu")
    camada_total_connect2 = keras.layers.Dense(50, activation="relu")
    camada_total_connect3 = keras.layers.Dense(5, activation="softmax")
    
    input = keras.Input(shape + (3,))

    network = preprocess_input(input)

    network = model(network, training=False)

    network = global_average_layer(network)

    network = keras.layers.Dropout(rate=0.2)(network)

    output = camada_total_connect1(network)
    output = camada_total_connect2(output)
    output = camada_total_connect3(output)

    return keras.Model(input, output)

In [100]:
denseNet = buildModel(denseNet201)

In [101]:
denseNet.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 250, 250, 3)]     0         
                                                                 
 tf.math.truediv_20 (TFOpLam  (None, 250, 250, 3)      0         
 bda)                                                            
                                                                 
 tf.nn.bias_add_10 (TFOpLamb  (None, 250, 250, 3)      0         
 da)                                                             
                                                                 
 tf.math.truediv_21 (TFOpLam  (None, 250, 250, 3)      0         
 bda)                                                            
                                                                 
 densenet201 (Functional)    (None, 7, 7, 1920)        18321984  
                                                          

In [107]:
denseNet.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.01),
    metrics=metrics,
    loss=keras.losses.SparseCategoricalCrossentropy(),
)

In [108]:
historic_normal = denseNet.fit(
    x=train_dataset,
    validation_data=validation_dataset,
    epochs=20,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(
            factor=0.001,
            verbose=1,
        ),
    ],
    verbose=1,
)

Epoch 1/20
235/235 [==============================] - 66s 210ms/step - loss: 0.0642 - accuracy: 0.9839 - precision: 1.0000 - kappa: 0.0000e+00 - val_loss: 0.0299 - val_accuracy: 0.9960 - val_precision: 1.0000 - val_kappa: 0.0000e+00 - lr: 0.0100
Epoch 2/20
235/235 [==============================] - 48s 202ms/step - loss: 0.0635 - accuracy: 0.9839 - precision: 1.0000 - kappa: 0.0000e+00 - val_loss: 0.0334 - val_accuracy: 0.9960 - val_precision: 1.0000 - val_kappa: 0.0000e+00 - lr: 0.0100
Epoch 3/20
235/235 [==============================] - 46s 195ms/step - loss: 0.0382 - accuracy: 0.9881 - precision: 1.0000 - kappa: 0.0000e+00 - val_loss: 0.0253 - val_accuracy: 0.9960 - val_precision: 1.0000 - val_kappa: 0.0000e+00 - lr: 0.0100
Epoch 4/20
235/235 [==============================] - 45s 190ms/step - loss: 0.0481 - accuracy: 0.9869 - precision: 1.0000 - kappa: 0.0000e+00 - val_loss: 0.0354 - val_accuracy: 0.9920 - val_precision: 1.0000 - val_kappa: 0.0000e+00 - lr: 0.0100
Epoch 5/20
235/2

In [ ]:
denseNet201.trainable = True

for i in denseNet201.layers[:100]:
    i.trainable = False

In [ ]:
denseNet.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.0000001),
    metrics=metrics,
    loss=keras.losses.SparseCategoricalCrossentropy(),
)

In [ ]:
historic_fine_tunning = denseNet.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=30,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(
            factor=0.005,
            verbose=1,
        ),
    ],
    verbose=1,
    initial_epoch=20,
)

Epoch 21/30
235/235 [==============================] - 161s 545ms/step - loss: 0.0197 - accuracy: 0.9949 - val_loss: 0.0433 - val_accuracy: 0.9980 - lr: 1.0000e-07
Epoch 22/30
235/235 [==============================] - 121s 514ms/step - loss: 0.0164 - accuracy: 0.9949 - val_loss: 0.0424 - val_accuracy: 0.9980 - lr: 1.0000e-07
Epoch 23/30
235/235 [==============================] - 121s 513ms/step - loss: 0.0094 - accuracy: 0.9960 - val_loss: 0.0423 - val_accuracy: 0.9980 - lr: 1.0000e-07
Epoch 24/30
235/235 [==============================] - 121s 515ms/step - loss: 0.0092 - accuracy: 0.9967 - val_loss: 0.0426 - val_accuracy: 0.9980 - lr: 1.0000e-07
Epoch 25/30
235/235 [==============================] - 121s 513ms/step - loss: 0.0076 - accuracy: 0.9979 - val_loss: 0.0419 - val_accuracy: 0.9980 - lr: 1.0000e-07
Epoch 26/30
235/235 [==============================] - 121s 515ms/step - loss: 0.0086 - accuracy: 0.9975 - val_loss: 0.0424 - val_accuracy: 0.9980 - lr: 1.0000e-07
Epoch 27/30
235/

In [ ]:
denseNet.save("/content/drive/MyDrive/denseNet.h5")

In [ ]:
denseNet2 = keras.models.load_model("/content/drive/MyDrive/denseNet.h5")


In [ ]:
classes = ('Arborio', 'Basmati', 'Ipsala', 'Jasmine', 'Karacadag')
classes

('Arborio', 'Basmati', 'Ipsala', 'Jasmine', 'Karacadag')

In [ ]:
test_dataset = []
y_true = []

for cl in classes:
    imgs_path = glob(f"{test_path}/{cl}/*")

    for img_path in imgs_path:
        test_dataset.append(
            imread(img_path)
        )

        y_true.append(
            classes.index(cl)
        )

In [ ]:
test_dataset = np.array(test_dataset)
y_true = np.array(y_true)
y_true

array([0, 0, 0, ..., 4, 4, 4])

In [ ]:
y_pred = denseNet2.predict(test_dataset)

63/63 [==============================] - 11s 169ms/step


In [ ]:
y_pred = np.argmax(y_pred, axis=1)
y_pred

array([0, 0, 0, ..., 4, 4, 4])

In [ ]:
accuracy_score(y_pred, y_true)

0.998

In [ ]:
cohen_kappa_score(y_pred, y_true)

0.9975

In [ ]:
precision_score(y_pred, y_true, average='weighted')

0.9980025